In [1]:
import requests
import json
import os

# mettre son appID et sa key puis supprimer.
appid = "a5b42571"
key = ""
url = "http://api.adzuna.com/v1/api/jobs/fr/categories?app_id={0}&app_key={1}&&content-type=application/json".format(appid, key)
req1 = requests.get(url).json()

categories = req1["results"]


tags = [] # j'initie cette liste qui récupèrera les tags qui correspondent aux catégories

for cat in categories:
    tags.append(cat["tag"]) # Pour chaque catégorie je récupère son tag

print(tags)

['accounting-finance-jobs', 'it-jobs', 'sales-jobs', 'customer-services-jobs', 'engineering-jobs', 'hr-jobs', 'healthcare-nursing-jobs', 'hospitality-catering-jobs', 'pr-advertising-marketing-jobs', 'logistics-warehouse-jobs', 'teaching-jobs', 'trade-construction-jobs', 'admin-jobs', 'legal-jobs', 'creative-design-jobs', 'graduate-jobs', 'retail-jobs', 'consultancy-jobs', 'manufacturing-jobs', 'scientific-qa-jobs', 'social-work-jobs', 'travel-jobs', 'energy-oil-gas-jobs', 'property-jobs', 'charity-voluntary-jobs', 'domestic-help-cleaning-jobs', 'maintenance-jobs', 'part-time-jobs', 'other-general-jobs', 'unknown']


Connexion à la base de données

In [2]:
from sqlalchemy import create_engine, MetaData, text

username = "user"
password = "password"
hostname = "52.31.100.105"
port = "3306"
database_name = "database"

db_url = f"mysql+mysqlconnector://{username}:{password}@{hostname}:{port}/{database_name}"
metadata = MetaData()
engine = create_engine(db_url)



Création de la base de données

In [21]:
with engine.connect() as connection:
    connection.execute(text("CREATE TABLE POSTES (nom_poste varchar(255),date_creation datetime,ville varchar(255),pays varchar(255),url varchar(255),nom_entreprise varchar(255),id bigint,salaire int,type_contrat varchar(255),categorie varchar(255),latitude float,longitude float);"))
    

Récupération des données

In [40]:
countries = ["fr"]
results, success, failed = [], [], [] # Création de 3 listes, une pour les résultats, une pour la journalisation des succés et des échecs
for country in countries:
    for page in range(30, 90):
        url = "https://api.adzuna.com/v1/api/jobs/{3}/search/{0}?app_id={1}&app_key={2}".format(page, appid, key, country)
        req = requests.get(url)
        if req.status_code == 200: # Si réponse positive, alors on récupère les données
            try:
                data = req.json()
                results.append(data["results"])
            except:
                failed.append("{} : {}".format(country, page))
                print(req.status_code, " pour la page ", page, "du pays ", country)
            else:
                success.append("{} : {}".format(country, page))

        else :
            print(req.status_code, " pour la page ", page, "du pays ", country)

print(f"Succès : {len(success)}\nEchec : {len(failed)}")

Succès : 60
Echec : 0


Importation des données

In [42]:
success_insert, failed_insert = 0, 0

with engine.connect() as connection:
    for page in results:
        for annonce in page:
            nom_poste = annonce['title']
            date_creation = str(annonce['created']).replace("T", " ").replace("Z", "")
            ville = annonce['location']['display_name']
            pays = annonce['location']['area'][0]
            url = annonce['redirect_url']
            try:
                nom_entreprise = annonce['company']['display_name']
            except:
                nom_entreprise = "inconnu"
            id = annonce['id']
            salaire = annonce['salary_is_predicted']
            try:
                type_contrat = annonce['contract_type']
            except:
                type_contrat = "inconnu"
            categorie = annonce['category']['label']
            try:
                latitude = annonce['latitude']
            except:
                latitude = ""
            try:
                longitude = annonce['longitude']
            except:
                longitude = ""
            
            # On ajoute la ligne à la base de données
            try:
                connection.execute(text("INSERT INTO POSTES (nom_poste, date_creation, ville, pays, url, nom_entreprise, id, salaire, type_contrat, categorie, latitude, longitude) VALUES (:nom_poste, :date_creation, :ville, :pays, :url, :nom_entreprise, :id, :salaire, :type_contrat, :categorie, :latitude, :longitude)"), {"nom_poste": nom_poste, "date_creation": date_creation, "ville": ville, "pays": pays, "url": url, "nom_entreprise": nom_entreprise, "id": id, "salaire": salaire, "type_contrat": type_contrat, "categorie": categorie, "latitude": latitude, "longitude": longitude})
            except:
                failed_insert += 1
            else:
                success_insert += 1
            
    connection.commit()       
    resultat = connection.execute(text("SELECT * FROM POSTES;"))
    print(f"Succès : {success_insert}\nEchec : {failed_insert}")
    print(resultat.fetchall())

Succès : 468
Echec : 132
[('Auxiliaire de vie H/F à Mâcon', datetime.datetime(2024, 4, 4, 2, 15, 40), 'Saône-et-Loire, Bourgogne-Franche-Comté', 'France', 'https://www.adzuna.fr/details/4635318128?utm_medium=api&utm_source=a5b42571', 'Petits-fils', 4635318128, 0, 'permanent', 'Emplois Soins de santé et infirmiers', 46.6512, 4.71344), ('Auxiliaire de vie H/F à Bayeux', datetime.datetime(2024, 4, 4, 2, 15, 41), 'Tilly-sur-Seulles, Caen', 'France', 'https://www.adzuna.fr/details/4635318248?utm_medium=api&utm_source=a5b42571', 'Petits-fils', 4635318248, 0, 'permanent', 'Emplois Soins de santé et infirmiers', 49.2258, -0.63501), ('Auxiliaire de vie H/F à Montreuil', datetime.datetime(2024, 4, 4, 10, 15, 34), 'Montreuil, Bobigny', 'France', 'https://www.adzuna.fr/details/4635492456?utm_medium=api&utm_source=a5b42571', 'Petits-fils', 4635492456, 0, 'contract', 'Emplois Soins de santé et infirmiers', 48.8583, 2.43684), ('Business Developer indépendant(e) (H/F)', datetime.datetime(2024, 3, 11, 